# ID2214/FID3214 Assignment 2 Group no. [enter]
### Project members: 
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [5]:
import numpy as np
import pandas as pd
import time

## Reused functions from Assignment 1

In [6]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

from pandas.api.types import CategoricalDtype


def create_column_filter(df):
    # List of columns we need to consider.
    # Use sets to easily subtract ID and CLASS from the list of columns.
    subset = list(set(df.columns.tolist())-{"ID", "CLASS"})

    # List of columns to keep
    column_filter = ["ID", "CLASS"]

    # Only select columns that contain at least 2 unique values.
    for col in subset:
        if df[col].nunique() > 1:
            column_filter.append(col)

    return apply_column_filter(df, column_filter), column_filter


def apply_column_filter(input, column_filter):
    # Intersection of columns in input and columns to filter.
    keep_columns = list(set(input.columns.tolist()).intersection(set(column_filter)))
    return input[keep_columns]


def create_normalization(df, normalizationtype="minmax"):
    # List of columns we need to consider.
    # Use sets to easily subtract ID and CLASS from the list of columns.
    subset = list(set(df.columns.tolist())-{"ID", "CLASS"})

    # zip 3 lists together: A list of the type, the minimums of the columns, the maximums of the colums
    if normalizationtype == "minmax":
        norms = list(zip(["minmax" for _ in range(len(subset))], df[subset].min(), df[subset].max()))
    elif normalizationtype == "zscore":
        norms = list(zip(["zscore" for _ in range(len(subset))], df[subset].mean(), df[subset].std()))
    else:
        raise ValueError("wrong normalization type")

    # Combine the columns with their respective values, and convert to a dict.
    normalization = dict(zip(subset, norms))

    return apply_normalization(df, normalization), normalization


def apply_normalization(input_df, normalization):
    df = input_df.copy()

    # loop through the dictionaly, and apply the repsective normalization rowwise.
    for col, norm in normalization.items():
        if norm[0] == "minmax":
            df[[col]] = (df[[col]]-norm[1])/(norm[2]-norm[1])
        if norm[0] == "zscore":
            df[[col]] = (df[[col]]-norm[1])/(norm[2])

    return df


def create_imputation(df):
    # List of columns we need to consider.
    # Use sets to easily subtract ID and CLASS from the list of columns.
    subset = list(set(df.columns.tolist())-{"ID", "CLASS"})

    numeric_means = df[subset].mean(numeric_only=True).to_dict()
    # for catagoric (non-number) columns, calculate the modes and convert to dict.
    catagoric_modes = df[subset].select_dtypes(exclude=[np.number]).mode().iloc[0].to_dict()

    imputation = {**numeric_means, **catagoric_modes}
    return apply_imputation(df, imputation), imputation


def apply_imputation(input_df, imputation):
    df = input_df.copy()
    df = df.fillna(imputation)
    return df


def create_bins(df, nobins, bintype="equal-width"):
    # List of columns we need to consider, only numeric.
    # Use sets to easily subtract ID and CLASS from the list of columns.
    subset = list(set(df.select_dtypes(include=np.number).columns.tolist())-{"ID", "CLASS"})

    df = df.copy()
    binning = dict()
    # For every column, call the respective pandas function for binning.
    for col in subset:
        if bintype == "equal-width":
            df[col], binning[col] = pd.cut(df[col], nobins, retbins=True, duplicates='drop', labels=False)
        if bintype == "equal-size":
            df[col], binning[col] = pd.qcut(df[col], nobins, retbins=True, duplicates='drop', labels=False)

        # Convert boundaries of outer bins to infinite.
        binning[col][0] = -float('inf')
        binning[col][-1] = float('inf')

    return df, binning


def apply_bins(df, binning):
    df = df.copy()
    for col, bins in binning.items():
        df[col] = pd.cut(df[col], bins, labels=False)
    return df


def create_one_hot(df):
    # List of columns we need to consider, only numeric.
    # Use sets to easily subtract ID and CLASS from the list of columns.
    subset = list(set(df.select_dtypes(include=['object', 'category']).columns.tolist())-{"ID", "CLASS"})

    # Create dict of the possible values for each column
    one_hot = dict()
    for col in subset:
        one_hot[col] = df[col].dropna().unique()

    return apply_one_hot(df, one_hot), one_hot


def apply_one_hot(df, one_hot):
    df = df.copy()
    for col, values in one_hot.items():
        # convert values to categorical
        df[col] = df[col].astype(CategoricalDtype(values))
        # Use get_dummies to get one-hot encoding.
        dummies = pd.get_dummies(df[col], prefix=col, dtype=float)

        # Add new columns to dataframe, and drop the old column.
        df = pd.concat([df, dummies], axis=1)
        df.drop([col], axis=1, inplace=True)

    return df


def split(df, testfraction):
    df_split = df.copy()
    if testfraction > 1 or testfraction < 0:
        raise ValueError("testfraction must be between 0 and 1")

    # Calculate test length
    test_frac = int(df_split.shape[0] * testfraction)
    # Calculate random test indexes of length test_frac
    test_idx = np.random.permutation(df_split.shape[0])[:test_frac]

    # Create dataset based on test_idx
    trainingdf = df_split.drop(test_idx)
    testdf = df_split.loc[test_idx]

    return trainingdf, testdf


def accuracy(df, correctlabels):
    # Get the indexex of the columns with the highest values. Take the first if they are the same.
    predictions = df.idxmax(axis="columns")
    # Ternary expection to get the total amount of correct predictions.
    correct = sum(label == guess for label, guess in zip(correctlabels, predictions))

    return correct/len(predictions)


def folds(input_df, nofolds=10):
    if nofolds <= 1:
        raise ValueError("nofolds should be greater than 1")
    # Randum shuffle by using a sample of fraction 1.
    df = input_df.sample(frac=1).reset_index(drop=True)
    # Use np.array_split to split into nofolds.
    return np.array_split(df, nofolds)


def brier_score(df, correctlabels):
    labels_vec = np.zeros([len(correctlabels), df.shape[1]])
    # Converts labels into one-hot encoded vectors
    for i, l in enumerate(correctlabels):
        labels_vec[i] = np.where(df.columns == l, 1, 0)

    values = df.copy().values

    # Calculate mean squared error.
    brier_score = np.sum(np.power(values - labels_vec, 2)) / values.shape[0]

    return brier_score


def auc(df: pd.DataFrame, correctlabels: []):
    auc_tot = 0
    for c in df.columns.tolist():
        class_labels = np.where(np.array(correctlabels)==c, True, False)
        weight = np.sum(class_labels) / class_labels.shape[0]

        tp = np.zeros(class_labels.shape[0])
        fp = np.zeros(class_labels.shape[0])
        for i, v in enumerate(df[c].values):
            if class_labels[i] == True:
                tp[i] = 1
            else:
                fp[i] = 1

        scores = np.zeros([len(tp),3])
        for i in range(len(tp)):
            scores[i] = [df[c][i], tp[i], fp[i]]
        
        # sort in reverse
        scores = scores[scores[:,0].argsort()[::-1]]
        
        auc = 0
        cov_tp = 0
        tot_tp = np.sum(tp)
        tot_fp = np.sum(fp)
        for i in range(scores.shape[0]):
            tp_i = scores[i][1]
            fp_i = scores[i][2]
            
            if fp_i == 0: # no false positives
                cov_tp += tp_i
            elif tp_i == 0: # no true positives
                auc += (cov_tp/tot_tp)*(fp_i/tot_fp)
            else:
                auc += (cov_tp/tot_tp)*(fp_i/tot_fp) + (tp_i/tot_tp)*(fp_i/tot_fp)/2
                cov_tp += tp_i
        
        auc_tot += auc * weight
    
    return auc_tot


## 1. Define the class kNN

In [7]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies

class kNN():
    def __init__(self):
        self.column_filter, self.imputation, self.one_hot, self.labels, self.training_labels, self.training_data, self.training_time = None, None, None, None, None, None, None

    def fit(self, df, normalizationtype = 'minmax'):
        # The list of labels from the training set.
        self.training_labels = df['CLASS'].astype('category')
        # The list of possible labels.
        self.labels = self.training_labels.cat.categories.tolist()

        # Take only the feature columns from the training data.
        features = list(set(df.columns.tolist())-{"ID", "CLASS"})
        df = df[features]
        # Preprocessing
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.normalization = create_normalization(df, normalizationtype = normalizationtype)
        df, self.one_hot = create_one_hot(df)
        self.training_data = df
        return

    def predict(self, df, k):
        # Take only the feature columns from the training data.
        features = list(set(df.columns.tolist())-{"ID", "CLASS"})
        df = df[features]

        # Aply preprocessing, with fitted configuration.
        df = apply_column_filter(df, self.column_filter)
        df = apply_imputation(df, self.imputation)
        df = apply_normalization(df, self.normalization)
        df = apply_one_hot(df, self.one_hot)

        # Initialize empty dataframe for the predictions
        predictions = pd.DataFrame([], columns=self.labels)
        for i, point in df.iterrows():
            # Calculate the distance from this point to every point in the training set.
            distances = self.training_data.apply(lambda row: euclidean_dist(row, point), axis=1)
            # Combine the distances with their indices in a dataframe.
            neighbors = pd.DataFrame(data={"dist": distances, "idx": distances.index})
            # Sort on closest distance
            neighbors = neighbors.sort_values(by=['dist'])

            # Dictionary with the votes for each label.
            label_count = {}
            # For every of the closest k neigbours
            for j, neighbor in neighbors.head(k).iterrows():
                # Add one vote to the label of the neighbour. Initialize if its the first vote for the label.
                if self.training_labels[neighbor['idx']] in label_count:
                    label_count[self.training_labels[neighbor['idx']]] += 1
                else:
                    label_count[self.training_labels[neighbor['idx']]] = 1
            # Append the list with votes to the final dataframe.
            predictions = predictions.append(label_count, ignore_index=True)

        # Fill empty values with 0, then normalize to get the class probabilities.
        predictions = predictions.fillna(0)
        predictions = predictions.div(k, axis=0)
        return predictions

def euclidean_dist(a, b):
    dist = 0
    for c in range(len(a)):
        dist += (a[c] - b[c]) ** 2
    return np.sqrt(dist)


In [8]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time: 0.09 s.
Testing time (k=1): 1.09 s.
Testing time (k=3): 1.13 s.
Testing time (k=5): 1.21 s.
Testing time (k=7): 1.22 s.
Testing time (k=9): 1.27 s.



'results'

,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.805011
3,0.663551,0.488058,0.828083
5,0.579439,0.474019,0.839388
7,0.598131,0.470723,0.833960
9,0.616822,0.483674,0.832325


In [9]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 1.0000
Brier score on training set (k=1): 0.0000


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [10]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical) 
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical) 
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors

class NaiveBayes():
    def __init__(self):
        return

    def fit(self, df, nobins=10 , bintype='equal-width'):
        # Convert the column to category dtype
        df['CLASS'] =  df['CLASS'].astype('category')
        # List of possible labels
        self.labels = df['CLASS'].cat.categories.tolist()
        # List of feature names
        features = list(set(df.columns.tolist())-{"ID", "CLASS"})

        # Preprocessing
        df, self.column_filter = create_column_filter(df)
        df, self.binning = create_bins(df, nobins, bintype=bintype)

        # Calculate the normalized class priors
        self.class_priors = df["CLASS"].value_counts(normalize=True).to_dict()

        # Calculate the feature value counts for each class.
        self.feature_class_value_counts = dict()
        for feature in features:
            class_value_counts = df.groupby('CLASS')[feature].value_counts().to_dict()
            self.feature_class_value_counts[feature] = class_value_counts

        # Calculate the class counts (per non-null feature).
        self.feature_class_counts = dict()
        for feature in features:
            class_counts = df.groupby('CLASS')[feature].count().to_dict()
            self.feature_class_counts[feature] = class_counts


    def predict(self, df):
        # Apply proprocessing from fitted configuration
        df = apply_column_filter(df, self.column_filter)
        df = apply_bins(df, self.binning)

        # Final list with class probabilities
        predictions = pd.DataFrame([], columns=self.labels)
        for i, point in df.iterrows():
            # Dictionary with probabilites per class for this point
            probabilities = dict()
            for c, prior in self.class_priors.items():
                # Prior for this class probability
                probabilities[c] = prior
                for feature, class_counts in self.feature_class_counts.items():
                    # For each feature. Calculate the respective a-priori, and add it to the total probability.
                    key = (c, point[feature])
                    class_count = class_counts[c] if c in class_counts else 0
                    class_value_count = self.feature_class_value_counts[feature][key] if key in self.feature_class_value_counts[feature] else 0

                    probabilities[c] *= class_value_count / class_count
            # Add probabilites for this point to the dataframe
            predictions = predictions.append(probabilities, ignore_index=True)
        
        # Fill empty values with 0, then normalize to get the class probabilities.
        predictions = predictions.fillna(0)
        predictions = predictions.div(df.sum(axis=1), axis=0)
        # If the sum is 0, replace with priors
        for i, pred in predictions.iterrows():
            if np.sum(pred) == 0:
               predictions.iloc[i] = self.class_priors
        return predictions        


In [11]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.05 s.
Testing time (3, 'equal-width'): 0.43 s.
Training time (3, 'equal-size'): 0.03 s.
Testing time (3, 'equal-size'): 0.32 s.
Training time (5, 'equal-width'): 0.05 s.
Testing time (5, 'equal-width'): 0.35 s.
Training time (5, 'equal-size'): 0.04 s.
Testing time (5, 'equal-size'): 0.40 s.
Training time (10, 'equal-width'): 0.08 s.
Testing time (10, 'equal-width'): 0.37 s.
Training time (10, 'equal-size'): 0.03 s.
Testing time (10, 'equal-size'): 0.29 s.



'results'

Accuracy  Brier score       AUC
3  equal-width  0.616822     1.000269  0.681596
   equal-size   0.607477     0.999979  0.758497
5  equal-width  0.644860     0.991682  0.650165
   equal-size   0.598131     0.999999  0.763702
10 equal-width  0.654206     0.957292  0.653723
   equal-size   0.588785     0.997831  0.772095

In [12]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.8505
AUC on training set: 0.8803
Brier score on training set: 1.0000


### Comment on assumptions, things that do not work properly, etc.